In [ ]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.image import resize
from tensorflow.keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -q '/content/drive/MyDrive/Dataset/Audio_dataset.zip' -d '/content/'

In [ ]:
# Define your folder structure
data_dir = 'Audio_dataset'
classes = ['Danger', 'Normal']

# Load and preprocess audio data
def load_and_preprocess_data(data_dir, classes, target_shape=(256, 256)):
    data = []
    labels = []

    for i, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.mp3'):
                file_path = os.path.join(class_dir, filename)
                audio_data, sample_rate = librosa.load(file_path, sr=None)
                # Perform preprocessing (e.g., convert to Mel spectrogram and resize)
                mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
                mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
                data.append(mel_spectrogram)
                labels.append(i)

    return np.array(data), np.array(labels)

# Split data into training and testing sets
data, labels = load_and_preprocess_data(data_dir, classes)
labels = to_categorical(labels, num_classes=len(classes))  # Convert labels to one-hot encoding
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Create a neural network model
input_shape = X_train[0].shape
input_layer = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
output_layer = Dense(len(classes), activation='softmax')(x)
model = Model(input_layer, output_layer)

<ipython-input-3-7fbc7e459186>:15: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sample_rate = librosa.load(file_path, sr=None)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-3-7fbc7e459186>:15: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sample_rate = librosa.load(file_path, sr=None)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-3-7fbc7e459186>:15: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sample_rate = librosa.load(file_path, sr=Non

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
10/10 [==============================] - 19s 2s/step - loss: 0.0219 - accuracy: 0.9968 - val_loss: 0.6982 - val_accuracy: 0.9231
Epoch 2/50
10/10 [==============================] - 18s 2s/step - loss: 0.0608 - accuracy: 0.9968 - val_loss: 0.1217 - val_accuracy: 0.9487
Epoch 3/50
10/10 [==============================] - 18s 2s/step - loss: 0.0263 - accuracy: 0.9968 - val_loss: 0.3030 - val_accuracy: 0.9487
Epoch 4/50
10/10 [==============================] - 18s 2s/step - loss: 0.0338 - accuracy: 0.9968 - val_loss: 0.1296 - val_accuracy: 0.9487
Epoch 5/50
10/10 [==============================] - 18s 2s/step - loss: 0.0338 - accuracy: 0.9968 - val_loss: 0.1313 - val_accuracy: 0.9487
Epoch 6/50
10/10 [==============================] - 18s 2s/step - loss: 0.0303 - accuracy: 0.9968 - val_loss: 0.1466 - val_accuracy: 0.9615
Epoch 7/50
10/10 [==============================] - 18s 2s/step - loss: 0.0161 - accuracy: 0.9968 - val_loss: 1.0930 - val_accuracy: 0.9231
Epoch 8/50
10/10 [==

In [ ]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy)

[0.7267650961875916, 0.9230769276618958]


In [ ]:
model.save('audio_classification_model.h5')

In [ ]:
# Save the model
model.save('audio_classification_model.h5')
# ovide a function to test the trained model on new audio files. This function loads an audio file, preprocesses it, makes predictions, and displays the predicted class along with class probabilities. This is a crucial step in applying the model to real-world scenarios.

# Load the saved model
model = load_model('audio_classification_model.h5')

# Define the target shape for input spectrograms
target_shape = (256, 256)

# Define your class labels
classes = ['Danger', 'Normal']

# Function to preprocess and classify an audio file
def test_audio(file_path, model):
    # Load and preprocess the audio file
    audio_data, sample_rate = librosa.load(file_path, sr=None)
    mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
    mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
    mel_spectrogram = tf.reshape(mel_spectrogram, (1,) + target_shape + (1,))

    # Make predictions
    predictions = model.predict(mel_spectrogram)

    # Get the class probabilities
    class_probabilities = predictions[0]

    # Get the predicted class index
    predicted_class_index = np.argmax(class_probabilities)

    return class_probabilities, predicted_class_index

# Test an audio file
test_audio_file = '/content/check_danger2.mp3'
class_probabilities, predicted_class_index = test_audio(test_audio_file, model)

# Display results for all classes
for i, class_label in enumerate(classes):
    probability = class_probabilities[i]
    print(f'Class: {class_label}, Probability: {probability:.4f}')

# Calculate and display the predicted class and accuracy
predicted_class = classes[predicted_class_index]
accuracy = class_probabilities[predicted_class_index]
print(f'The audio is classified as: {predicted_class}')
print(f'Accuracy: {accuracy:.4f}')

1/1 [==============================] - 0s 89ms/step
Class: Danger, Probability: 1.0000
Class: Normal, Probability: 0.0000
The audio is classified as: Danger
Accuracy: 1.0000
